TO DO:
    - generate numbers 
    -think aboout visualization
    -chanke how random chosen based on data
    -release time zero
    
  
THINK ABOUT:
   -rigth time step to check
   -Make the algorithm go on parallel or one after the other
   -Note: DSOS: load at the moment--> yes otherwise eliminated from list
   -what happen if expected tìprocessing time=0

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
from bisect import insort_left,insort_right
from tabulate import tabulate
from random import randint

In [2]:
class Job_b:
    def __init__(self, job_id, release, processing,expected,CV, weigth,alpha=None,MBT=0):
        # information needed for single machine: id, release, expected processing time, weigths
        self.id = job_id
        self.release = release
        self.processing = processing
        self.expected=expected
        self.CV=CV
        self.we=weigth
        self.alpha_RSOS=None
        self.alpha_DSOS=None
        self.ta_RSOS=None
        self.ta_DSOS=None
        self.done_1=0
        self.MBT=0
        self.comp_RSOS=0
        self.comp_DSOS=0


In [3]:
def check_arrival_time(jobs,start_time, last_check_time):
    current_time = datetime.now()
    elapsed_seconds = (current_time - start_time).total_seconds()

    new_arrivals = []
    if last_check_time==0: 
        for idx, job in enumerate(jobs):
            if job.release <= elapsed_seconds:
                new_arrivals.append(idx)
    else: 
        for idx, job in enumerate(jobs):
        #print("elapsed_seconds", elapsed_seconds, "last_check_time", last_check_time)
            if job.release <= elapsed_seconds  and job.release > last_check_time:
                new_arrivals.append(idx)

    return new_arrivals

In [27]:
def run_one_both(jobs, m,alpha,time_step):
    np.random.seed(10)
    
    
    # List to store arrived jobs
    L=[]
    upper_time=max(job.release for job in jobs)+sum(job.expected for job in jobs)
    machines_DSOS=np.zeros(m)
    machines_RSOS=np.zeros(m)
    ite=0

    # Initialize last check time
    relative_current = 0
    # Record the starting time of the algorithm
    start_time = datetime.now()

    # Check for arrival time every 1 second
    while sum(job.done_1 for job in jobs)<len(jobs):
        ite=ite+1
        new_arrivals = check_arrival_time(jobs,start_time, relative_current)
        # Update last check time
        relative_current = (datetime.now() - start_time).total_seconds()
        if Printing:
            if not new_arrivals:     
                print("\n\n NEW ITERATION")
                print("not new")
        #update L if new job arrived 
        for arrived_job_idx in new_arrivals:
            arrived_job = jobs[arrived_job_idx]
            arrived_job.done_1=0
            arrived_job.MBT=0
            if Printing: 
                print("\n\n NEW ITERATION")
                print(f"One job arrived at {datetime.now()} for {arrived_job.id}")

            ##MODYFY TO have 50 different cases for reproducibility
            #if RANDOM_alpha: 
             #   arrived_job.alpha=np.random.uniform(0,1)
            #else:
             #   arrived_job.alpha=alpha #MODYFY BASED  ON RSOS OR DSOS
            
            arrived_job.alpha_RSOS=np.random.uniform(0,1)
            #else:
            arrived_job.alpha_DSOS=alpha #MODYFY BASED  ON RSOS OR DSOS
            insort_right(L, arrived_job, key=lambda x: -(x.we/x.expected))

            #current_time = datetime.now()
            #relative_current=(current_time-start_time).total_seconds()

    
        #process first job in L
        if L:
            #print("list not empty")
            #print("start",len(L),L)
            remaning=(1-L[0].done_1)*L[0].expected/m 
            #print("for job::",L[0].id,"done:", L[0].done_1, "remaning:",remaning)
            #print("L",L, len(L))
            i=0
            tot=0
            while tot+remaning<=time_step and i <len(L):

                #L[i].MBT=L[i].MBT+(m/L[i].expected)*((current_time+timedelta(seconds=tot+remaning)).timestamp()**2-(current_time+timedelta(seconds=tot)).timestamp()**2)#correct
                #L[i].MBT=L[i].MBT+(m/L[i].expected)*((current_time-start_time+timedelta(seconds=tot+remaning)).total_seconds()**2-(current_time-start_time+timedelta(seconds=tot)).total_seconds()**2)
                L[i].MBT=L[i].MBT+(m/L[i].expected)*(remaning*(2*(tot+relative_current) +remaning))/2 # (relative_curreny+tot+remaing)^2-(relative_curreny+tot)^2
                if L[i].done_1< L[i].alpha_RSOS:
                    L[i].ta_RSOS=relative_current+tot+(L[i].alpha_RSOS-L[i].done_1)*L[i].expected/m
                    machine=randint(0,m-1) 
                        #machine=machines.index(min(machines))
                    if machines_RSOS[machine]>=L[i].ta_RSOS:
                        machines_RSOS[machine]=machines_RSOS[machine]+L[i].processing
                    else:
                        machines_RSOS[machine]=L[i].ta_RSOS+L[i].processing
                    L[i].comp_RSOS=machines_RSOS[machine]
                    
                if L[i].done_1< L[i].alpha_DSOS:
                    L[i].ta_DSOS=relative_current+tot+(L[i].alpha_DSOS-L[i].done_1)*L[i].expected/m
                    minimum_load=np.min(machines_DSOS)
                    machine=np.min(np.where(machines_DSOS==minimum_load))
                        #machine=machines.index(min(machines))
                    if machines_DSOS[machine]>=L[i].ta_DSOS:
                        machines_DSOS[machine]=machines_DSOS[machine]+L[i].processing
                    else:
                        machines_DSOS[machine]=L[i].ta_DSOS+L[i].processing
                    L[i].comp_DSOS=machines_DSOS[machine]

                L[i].done_1=1
                #current_time=current_time+(1-L[i].done_1)*L[i].expected/m 
                i=i+1
                tot=tot+remaning
                if len(L)>i:
                    remaning=(1-L[i].done_1)*L[i].expected/m 

            if len(L)>i:
                #print("i:" ,i ,"tot",tot )

                done_before=L[i].done_1    
                L[i].done_1=L[i].done_1+(time_step-tot)/(L[i].expected/m)
                #print('start',start,'end_time',end_time)
                #L[i].MBT=L[i].MBT+(m/L[i].expected)*((current_time+timedelta(seconds=time_step))**2-(current_time+timedelta(seconds=tot))**2)
                #L[i].MBT=L[i].MBT+(m/L[i].expected)*((end_time.timestamp())**2-(start.timestamp())**2)#correct
                #print("MBT", L[i].id,"before",L[i].MBT,"summed",(end_time**2-start**2),"after",L[i].MBT+(end_time**2-start**2))
                if done_before< L[i].alpha_RSOS<=L[i].done_1:
                    L[i].ta_RSOS=relative_current+tot+(L[i].alpha_RSOS-done_before)*L[i].expected/m 
                    machine=randint(0,m-1)
                    if machines_RSOS[machine]>=L[i].ta_RSOS:
                        machines_RSOS[machine]=machines_RSOS[machine]+L[i].processing
                    else:
                        machines_RSOS[machine]=L[i].ta_RSOS+L[i].processing
                    L[i].comp_RSOS=machines_RSOS[machine]
                if done_before< L[i].alpha_DSOS<=L[i].done_1:
                    L[i].ta_DSOS=relative_current+tot+(L[i].alpha_DSOS-done_before)*L[i].expected/m  
                    minimum_load=np.min(machines_DSOS)
                    machine=np.min(np.where(machines_DSOS==minimum_load))
                    if machines_DSOS[machine]>=L[i].ta_DSOS:
                        machines_DSOS[machine]=machines_DSOS[machine]+L[i].processing
                    else:
                        machines_DSOS[machine]=L[i].ta_DSOS+L[i].processing
                    L[i].comp_DSOS=machines_DSOS[machine]
                    
                    
                    
                L[i].MBT=L[i].MBT+(m/L[i].expected)*((relative_current+ time_step)**2-(relative_current+ tot)**2)/2
                #print("END","for job::",L[0].id,"done:", L[0].done_1, "remaning:",(1-L[0].done_1)*L[0].expected/m )
                L=L[i:]

            else: 
                L=[]
            

    # Print the table
        if Printing:
            #check alpha_point to see if it enters other list 
            table_data = [(job.id, job.release, job.done_1, (1-job.done_1)*job.expected/m, job.MBT , job.processing, job.alpha_RSOS, job.ta_RSOS, job.alpha_DSOS, job.ta_DSOS, job.comp_RSOS, job.comp_DSOS) for job in jobs]
            print("end",len(L),L)
            print("after iteration:", ite, "and time:", (datetime.now() - start_time).total_seconds())
            print(tabulate(table_data, headers=['Job ID','release', 'Done', 'Remaining', 'Mean busy time','processing','alpha_R','alpha_point_R','alpha_D','alpha_point_D','complition_R', 'complition_D']))
            print("RSOS",machines_RSOS)
            print("DSOS",machines_DSOS)


        time.sleep(time_step)  # sleep for 1 second
    total_comp_RSOS = sum(job.we * job.comp_RSOS for job in jobs)
    total_comp_DSOS = sum(job.we * job.comp_DSOS for job in jobs)
    total_LR= sum(job.we * (job.MBT-(job.CV-1)*job.expected/2) for job in jobs)
    return total_comp_RSOS,total_comp_DSOS, total_LR

In [15]:
jobs_data = [
    {'job_id': '1', 'release': 4, 'processing': 5, 'expected': 4.5, 'CV':3,'weigth':4.5},
    {'job_id': '2', 'release': 0, 'processing': 3,'expected': 3, 'CV':0,'weigth':3},
    {'job_id': '3', 'release': 3, 'processing': 5,'expected': 1.5, 'CV':1,'weigth':1.5},
    {'job_id': '4', 'release': 0, 'processing': 3,'expected': 3, 'CV':1,'weigth':3}
    #{'job_id': '5', 'release': 9, 'processing': 5,'expected': 1.5, 'CV':4,'weigth':1.5},
    #{'job_id': '6', 'release': 5, 'processing': 3,'expected': 3, 'CV':1,'weigth':3},
    #{'job_id': '7', 'release': 7, 'processing': 5,'expected': 1.5, 'CV':4,'weigth':1.5}


]
#convert list to list of object
jobs = [Job_b(**job_data) for job_data in jobs_data]

In [16]:
Ar=[job.release for job in jobs]
Ar.sort()
Ar
list(set(Ar)) 

[0, 3, 4]

In [29]:
time_step=1
m=3
alpha=0.5
RANDOM_alpha=True
RANDOM_assigment=True

Printing=True


In [30]:
jobs_after=run_one_both(jobs, m,alpha,time_step)




 NEW ITERATION
One job arrived at 2023-11-30 15:40:20.830838 for 1


 NEW ITERATION
One job arrived at 2023-11-30 15:40:20.830889 for 2
end 2 [<__main__.Job_b object at 0x125b67b90>, <__main__.Job_b object at 0x125b67710>]
after iteration: 1 and time: 0.000235
  Job ID    release      Done    Remaining    Mean busy time    processing    alpha_R  alpha_point_R      alpha_D    alpha_point_D    complition_R    complition_D
--------  ---------  --------  -----------  ----------------  ------------  ---------  ---------------  ---------  ---------------  --------------  --------------
       1          0  0.666667          0.5          0.333344             5  0.771321                          0.5         0.750016               0         5.75002
       2          0  0                 1            0                    3  0.0207519                         0.5                                0         0
       3          3  0                 0.5          0                    5                 

In [35]:

df= pd.DataFrame([vars(job) for job in jobs])
df_name=f"df_m_{m}"
globals()[df_name]=df
# Print the DataFrame
print(df_m_3)

  id  release  processing  expected  CV   we  alpha_RSOS  alpha_DSOS  \
0  1        0           5       4.5   3  4.5    0.771321         0.5   
1  2        0           3       3.0   0  3.0    0.020752         0.5   
2  3        3           5       1.5   1  1.5    0.633648         0.5   
3  4        7           3       3.0   1  3.0    0.748804         0.5   

    ta_RSOS   ta_DSOS  done_1       MBT  comp_RSOS  comp_DSOS  
0  1.160731  0.750016       1  0.751261   6.160731   5.750016  
1  1.524502  2.009980       1  2.006865   9.160731   5.009980  
2  3.334813  3.267989       1  3.267989   8.334813   8.267989  
3  7.793162  7.544358       1  7.544358  12.160731  10.544358  


In [345]:
print(jobs_after[0])

In [309]:
ordered_jobs = sorted(jobs, key=lambda job: job.ta)

In [34]:
# Calculate the sum over all jobs of the product between weight and completion using list comprehension
total_comp_R = sum(job.we * job.comp_RSOS for job in jobs)
total_comp_D= sum(job.we * job.comp_DSOS for job in jobs)
total_LR= sum(job.we * (job.MBT-(job.CV-1)*job.expected/2) for job in jobs)
#total_LR= sum(job.we * job.MBT for job in jobs)

print("Total complition & LPR:", total_comp_R, total_comp_D, total_LR)


Total complition & LPR: 104.18989480764594 84.9400695 21.1863255


In [215]:
m = 5  # Replace 5 with the desired number of lists

# Create m lists
lists = [[] for _ in range(m)]

# Access the lists by index
for i in range(m):
    lists[i].append(i)  # Appending the index for demonstration purposes

# Print the lists
for i, inner_list in enumerate(lists):
    print(f"List {i}: {inner_list}")


List 0: [0]
List 1: [1]
List 2: [2]
List 3: [3]
List 4: [4]


In [ ]:
#process first job in L
    current_time = datetime.now()
    tot=L[0].expected/m
    i=0
    while tot<time_step:
        L[i].done_1=1
        L[i].MBT=(m/L[i].expected)*((current_time+tot)^2-current_time^2)
        i=i+1
        tot=tot+(L[i].expected/m)

In [26]:
print(dir(L[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'alpha', 'complited1', 'complitedm', 'expected', 'id', 'processing', 'release', 'ta', 'we']


In [18]:
len(L)

3

In [46]:
L[0].done_1

29.0

In [34]:
from datetime import datetime, timedelta

# Get the current datetime
current_datetime = datetime.now()

# Add 5 minutes to the current datetime
new_datetime = current_datetime + timedelta(minutes=5)

# Print the original and new datetime values
print("Original Datetime:", current_datetime)
print("New Datetime (after adding 5 minutes):", new_datetime)
print(datetime.now())


Original Datetime: 2023-11-22 19:20:50.990968
New Datetime (after adding 5 minutes): 2023-11-22 19:25:50.990968
2023-11-22 19:20:50.991685


In [194]:
jobs[job.id=='5'].id

'5'

In [365]:
randint(0,2)

2

In [242]:
2>=3

False

In [245]:
machines=[2,0,0,0]
machines.index(min(machines))

1

In [259]:
machines=np.zeros(5)
machines[2:4]=-1
minimum_load=np.min(machines)
machine=np.min(np.where(machines==minimum_load))
machine

2

In [260]:
machines

array([ 0.,  0., -1., -1.,  0.])